# Select

## 기존 소켓 방식의 문제점

In [ ]:
import socket

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('127.0.0.1', 10000))
server_socket.listen(5)
client_socket, addr = server_socket.accept()

while True:
    data = client_socket.recv(1024)
    if data:
        client_socket.sendall(data)
    else:
        break

client_socket.close()
server_socket.close()

앞서 socket에서 설명한 코드는 문제점이 있는데, 소켓 API의 함수들이 Blocking I/O 모드로 동작한다는 점이다.

* `accept()`: 커넥션이 들어올때까지 **기다린다.**
* `connet()`: 커넥션이 연결될때까지 **기다린다.**
* `send()`: :데이터가 소켓의 버퍼에 들어갈때까지 **기다린다.**
* `recv()`: 패킷이 받아질때까지 **기다린다.**

즉, send, recv와 같은 함수들이 블로킹 상태가 되므로 여러 클라이언트와 동시에 데이터 송수신이 불가능하다. 

기존 소켓방식의 문제점을 해결하기위해, 다음과 같은 대안이 존재한다.

1. "멀티프로세스"기반 서버
2. "멀티스레딩"기반 서버
3. "멀티플렉싱"기반 서버

위의 방식을 사용하면 다수의 클라이언트와 통신이 가능해진다는것은 동일하나 차이점이 존재한다.

<br/>

### 멀티프로세스 기반 서버

![mp](./img/img_multi_process.png)


멀티프로세스 서버는 부모 프로세스가 소켓에서 accept(연결수립)시에<br/> 
자식 프로세스를 생성하고 클라이언트와 연결된 연결 소켓(소켓의 파일 디스크립터)을 자식 프로세스에게 전달하고 <br/>
실질적인 송수신에 대한 처리는 자식 프로세스가 담당한다.

다수의 프로세스를 사용하는 멀티프로세스 기반의 서버는 각 프로세스가 서로 간섭없이 독립적으로 수행된다는 장점이 있으나

프로세스는 상당히 비싼 자원이므로 프로세스의 빈번한 생성은 성능저하를 야기할 수 있다.

<br/>


### 멀티스레딩 기반 서버

![mt](./img/img_multi_thread.png)

멀티프로세스와 유사하게 연결이 생성될때마다 프로세스가 대신 스레드를 생성하는 방식이다.

메인 스레드가 소켓에서 accept시에 클라이언트와 연결된 연결 소켓(소켓의 파일 디스크립터)을 별도의 워커 스레드를 생성하여 넘겨준다.<br/>
실질적인 데이터 송수신처리는 워커 스레드에서 처리된다.<br/>

프로세스에 대한 비용보다 상대적으로 스레드 생성에 대한 리소스 비용이 적은 것은 맞으나 

멀티플렉싱을 사용하면 멀티스레딩을 사용하는 방식보다 훨씬 적은 비용으로 개별 클라이언트에 대한 요청을 처리할수 있다.



In [ ]:
def handle_client(conn, addr):
    with conn:
        data = conn.recv(1024)
        conn.sendall(data)
        
                
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    server_socket.bind(('127.0.0.1', 10000))
    server_socket.listen()

    while True:
        conn, addr = server_socket.accept()
        t = threading.Thread(target=handle_client, args=(conn, addr))
        t.start()

<br/>

### 멀티플렉싱 기반 서버
멀티플렉싱을 사용하면 각 클라이언트마다 별도의 스레드를 생성하는 것이 아닌<br/>
하나의 스레드에서 다수의 클라이언트에 연결된 연결 소켓(파일 디스크립터)을 관리하고<br>
소켓에 IO가 발생할때만 별도의 스레드를 만들어 해당 이벤트를 처리하도록 구현가능하다.

파이썬에서는 select 모듈을 사용하여 I/O 멀티플렉싱을 사용가능하다.

![mp](./img/img_select_model.png)

select 방식은 이벤트(입력, 출력, 에러)별로 감시할 파일들을 fd_set라는 파일 상태 테이블에 등록하고<br/>
등록된 파일(파일 디스크립터)에 어떤 이벤트가 발생했을 경우 fd_set를 확인하여 처리하는 방식으로 동작한다.

## Select를 적용한 소켓

In [ ]:
import socket
import select

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('127.0.0.1', 10000))
server_socket.listen()

readsocks = [server_socket]

while True:
    readables, writeables, excpetions = select.select(readsocks, [], [])
    for sock in readables:
        if sock == server_socket: # 신규 클라이언트 접속
            client_socket, addr = server_socket.accept()
            readsocks.append(client_socket)
            
        else: # 이미 접속한 클라이언트의 요청
            conn = sock
            data = conn.recv(1024)
            conn.sendall(data)
            conn.close()
            readsocks.remove(conn) # 클라이언트 접속 해제시 readsocks에서 제거

바뀐 소켓코드에서 핵심되는 부분은 다음 부분이다.

In [ ]:
readables, writeables, excpetions = select.select(readsocks, [], [])

해당 코드는 readsocks에 포함된 소켓에서 이벤트가 발생하는지 감시하는 역할을 수행하며 (이때는 블로킹)<br/>
감시하다가 readsocks에 속한 소켓에 이벤트가 발생하면 이후 코드가 실행된다.

select함수의 리턴값으로 받는 값들은 셋다 리스트이며 다음을 의미한다.
* `readables`: 수신한 데이터를 지니고 있는 소켓
* `writables`: 블로킹되지 않고 데이터의 전송이 가능한 소켓
* `exceptions`: 예외 상황이 발생한 소켓

위의 전체 소켓코드를 간략히 설명하면

1. `readsocks` 리스트에 서버소켓을 추가하여 select가 서버소켓에 클라이언트가 접속하는지를 감시시킨다.
2. 신규 클라이언트가 접속하면 `readsocks` 리스트에 신규 클라이언트의 소켓을 추가한다.
3. 신규 클라이언트에서 요청이 발생하면 else문이 수행되어 데이터 송수신 처리를 수행한다.